In [2]:
import fasttext
import numpy as np
import pandas as pd
import json
from googletrans import Translator
import copy
import math
from fuzzywuzzy import fuzz
import re


d:\hari files\project\wm_project\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
tamil_stop_words = []
with open("../Assets/TamilStopWords.txt", encoding='utf-8') as file:
    for line in file.readlines():
        tamil_stop_words.append(line.strip())

len(tamil_stop_words)

124

In [5]:
words_with_root = pd.read_csv(r"D:\hari files\project\wm_project\Assets\MostusedRootwords.csv")

In [6]:
tamil_nouns = []
with open("../Assets/all-tamil-nouns.txt", encoding='utf-8') as file:
    for line in file.readlines():
        tamil_nouns.append(line.strip())

len(tamil_nouns)

85256

In [7]:
tamil_verbs = []
with open("../Assets/Simple-verbs-01022021.txt", encoding='utf-8') as file:
    for line in file.readlines():
        tamil_verbs.append(line.strip())

len(tamil_verbs)

1461494

In [8]:
with open(r"D:\hari files\project\wm_project\Assets\eng_synonyms.json\eng_synonyms.json", 'r') as file:
    synonums = json.load(file)

In [9]:
opposites = pd.read_csv(r"D:\hari files\project\wm_project\Assets\opposites.csv")
opposites = opposites.drop(["Unnamed: 2"], axis = 1)
opposites.head()

,word1,word2
0,efficient,incompetent
1,dovish,hawkish
2,imaginary,real
3,lateral,bottom
4,invalid,valid


In [10]:
model_tamil = fasttext.load_model("D:\\hari files\\project\\wm_project\\Assets\\pretrained_word_to_vector\\cc.ta.300.bin\\cc.ta.300.bin")

In [11]:
def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [12]:
def find_root(w):
    for i in range(words_with_root.shape[0]):
        derivatives = words_with_root.iloc[i, 2].split(",")
        if w in derivatives:
            if w.endswith("ல்லை"):
                return (w,)
            return (words_with_root.iloc[i, 0],)
    return (w,)   

In [13]:
def remove_stop_words(_sentences):
    sentences = []
    for _sentence in _sentences:
        words = []
        for word in _sentence.split():
            if word not in tamil_stop_words:
                words.append(word)
        sentences.append(" ".join(word for word in words))
    return sentences

In [14]:
def convert_to_roots(_sentences):
    sentences = []
    for _sentence in _sentences:
        roots = []
        for _word in _sentence.split():
            root = find_root(_word)
            for r in root:
                roots.append(r)
        sentences.append(roots)
    return sentences

In [15]:
def is_dict_word(word):
    for i in range(words_with_root.shape[0]):
        if word == words_with_root.iloc[i, 0] or word in words_with_root.iloc[i, 2].split(","):
            return True
    return False

In [16]:
def get_proper_nouns(sentences):
    nouns = []
    for sentence in sentences:
        sentence_dup = []
        for word in sentence:
            if not is_dict_word(word) and word not in tamil_verbs:
                sentence_dup.append(word)
        nouns.append(sentence_dup)
    return nouns

In [17]:
def get_sentence_vectors(sentences):
    sentence_vectors = []
    for sentence in sentences:
        if sentence == []:
            sentence_vectors.append(None)
            continue
        
        sentence_vector = []
        for word in sentence.split():
            word_vector = model_tamil.get_word_vector(word)
            sentence_vector.append(word_vector)
        sentence_vectors.append(np.mean(sentence_vector, axis = 0))
    return sentence_vectors

def cosine_similarity(_vec1, _vec2):
    return np.dot(_vec1, _vec2) / (np.linalg.norm(_vec1) * np.linalg.norm(_vec2))

In [18]:
def translate_tamil_to_english(word):
    translator = Translator()
    translation = translator.translate(word, src='ta', dest='en')
    return translation.text

def find_synonym(w):
    for i in range(opposites.shape[0]):
        if(opposites.iloc[i, 0] == w):
            return w
    
    if w not in synonums:
        return w
    
    for synonum in synonums[w]:
        for i in range(opposites.shape[0]):
            if(opposites.iloc[i, 0] == synonum):
                return synonum
    return w

def is_opposite(_word1, _word2):
    flag1 = False
    flag2 = False

    if(_word1.endswith("ல்லை") or _word2.endswith("ல்லை")):
        if(_word1.endswith("ல்லை") and _word2[:-2] in _word1[:-6]):
            return True
        if(_word2.endswith("ல்லை") and _word1[:-2]in _word2[:-6]):
            return True

    print(_word1, _word2)
    _word1 = translate_tamil_to_english(_word1)
    _word2 = translate_tamil_to_english(_word2)
    w1 = find_synonym(_word1.lower())
    w2 = find_synonym(_word2.lower())

    for i in range(opposites.shape[0]):   
        if(opposites.iloc[i, 0] == w1):
            flag1 = True
        if(opposites.iloc[i, 0] == w2):
            flag2 = True

    if(flag1 and flag2):
        for i in range(opposites.shape[0]):
            if(
                (opposites.iloc[i, 0] in w1) and
                (opposites.iloc[i, 1] in w2)
            ):
                return True
        return False
    return False

In [19]:
def join_word_if_illai(input_list):
    new_list = copy.deepcopy(input_list)
    for j, inner_list in enumerate(input_list, 0):
        for i, word in enumerate(inner_list, 0):
            if word == 'இல்லை' and new_list[j][i - 1] is not None:
                new_list[j][i - 1] = inner_list[i - 1][:-1] + 'ில்லை'
                new_list[j].remove(word)
    return new_list

In [3]:
def remove_special_characters(text):
    sentences = []
    for sentence in text:
        pattern = r'[^\s\u0B80-\u0BFF]'
        clean_text = re.sub(pattern, '', sentence)
        sentences.append(clean_text)
    return sentences

In [53]:
# query = ["முதல் முறையாக ஆர்சிபி ஐபிஎல் கோப்பை வென்றது"]
# sentences = [
#     "ஆர்சிபி ஐபிஎல் கோப்பை வென்றதில்லை",
#     "இனி ராயல் சேலஞ்சர்ஸ் பெங்களூர் இல்லை ஆர்சிபி பெயர் ஜெர்ஸி அதிரடி மாற்றம்",
#     "ஆர்சிபி தொடர் தோல்விக்கு பின் தோல்வி",
# ]

# query = ["இந்தியா பிரதமர் நரேந்திரா மோடி ஊழல் காரணமாக கைது"]
# sentences = [
#     "பிரதமர் நரேந்திரா மோடி கருபுப்பணம் வழக்கில் சிக்கினார்",
#     "இந்தியா பிரதமர் இன்று காலை சிறை சென்றார்",
#     "பாஜக கட்சி மந்திரிகளுக்கு ஐந்து வருடம் சிறை தண்டனை",
# ]

# query = ["இது வரை கொரோனா வைரஸ் காரணமாக எந்த உயிரிழப்பும் இல்லை"]
# sentences = [
#                 "ஆபத்துடன் விளையாடும் சீன ஆய்வாளர்கள் உயிரிழப்பு நிச்சயம் பகீர் கிளப்பும் புது வகை கொரோனா",
#                 "கொரோனா வைரஸ் உங்கள் ஊரில் ஊரடங்கு நீடிக்கப்படுகிறதா",
#                 "கொரோனா தொற்று காரணமாக இன்று ஒரே நாளில் நான்கு பேர் உயிரிழப்பு",
#         ]

# query = ["முகேஷ் அம்பானி ஒரு பெண்ணுடன் புதிய திட்டத்திற்கு ஒப்புதல் அளித்துள்ளார்"]
# sentences = [
#                 "இந்திய மக்களுக்கான புதிய முதலீட்டுத் திட்டத்தை அறிவித்த முகேஷ் அம்பானி",
#                 "முகேஷ் அம்பானி, லைலா ராவ்ஸ் ஃபண்டில் முதலீடு செய்ய மக்களைக் கேட்டுக் கொண்டா",
#         ]

query = ["சந்திரயான் - 3 தென் துருவத்தில் தரையிறங்கியது"]
sentences = [
                "தென் துருவத்தில் தரையிறங்கும் இந்தியா",
                "சந்திரயான் - 3 நிலவின் தென் துருவத்தில் ஆக்சிஜன் கண்டுபிடிப்பு",
                "சந்திரயான்- 3 திட்டம் வெற்றி நிலவின் தென் துருவத்தில் கால்பதித்த முதல் நாடு இந்தியா",
                "அடுத்த வாரம் சந்திரயான் - 3 தென் துருவத்தில் தரையிறங்கத் தயாராகி வருகிறது"
        ]

# query = ["நடிகர் கமல் ஹாசன் இன்று ஆஸ்கார் விருது பெற்றார்"]
# sentences = [
#     "நடிகர் ரஜினிகாந்த் 2003 இன்று ஆஸ்கார் விருது பெற்றார்",
#     "ஆர்ஆர்ஆர் திரைபடத்திற்கு இந்த வருடம் ஆஸ்கார்",
#     "தென் இந்தியாவில் இருந்து இரண்டு நடிகர்கள் ஆஸ்கார் விருது பெற்றணர்"
# ]


In [54]:
query = remove_special_characters(query)
sentences = remove_special_characters(sentences)
print(query)
print(sentences)

['சந்திரயான்   தென் துருவத்தில் தரையிறங்கியது']
['தென் துருவத்தில் தரையிறங்கும் இந்தியா', 'சந்திரயான்   நிலவின் தென் துருவத்தில் ஆக்சிஜன் கண்டுபிடிப்பு', 'சந்திரயான்  திட்டம் வெற்றி நிலவின் தென் துருவத்தில் கால்பதித்த முதல் நாடு இந்தியா', 'அடுத்த வாரம் சந்திரயான்   தென் துருவத்தில் தரையிறங்கத் தயாராகி வருகிறது']


In [55]:
stop_free_query = remove_stop_words(query)
stop_free_sentences = remove_stop_words(sentences)
print(stop_free_query)
print(stop_free_sentences)

['சந்திரயான் தென் துருவத்தில் தரையிறங்கியது']
['தென் துருவத்தில் தரையிறங்கும் இந்தியா', 'சந்திரயான் நிலவின் தென் துருவத்தில் ஆக்சிஜன் கண்டுபிடிப்பு', 'சந்திரயான் திட்டம் வெற்றி நிலவின் தென் துருவத்தில் கால்பதித்த நாடு இந்தியா', 'வாரம் சந்திரயான் தென் துருவத்தில் தரையிறங்கத் தயாராகி வருகிறது']


In [56]:
stop_free_query_roots = convert_to_roots(stop_free_query)
stop_free_sentences_roots = convert_to_roots(stop_free_sentences)
print(stop_free_query_roots)
print(stop_free_sentences_roots)

[['சந்திரயான்', 'தென்', 'துருவம்', 'தரையிறங்கு']]
[['தென்', 'துருவம்', 'தரையிறங்கு', 'இந்தியா'], ['சந்திரயான்', 'நிலவு', 'தென்', 'துருவம்', 'ஆக்சிஜன்', 'கண்டுபிடிப்பு'], ['சந்திரயான்', 'திட்டம்', 'வெற்றி', 'நிலவு', 'தென்', 'துருவம்', 'கால்நடை', 'நாடு', 'இந்தியா'], ['வாரம்', 'சந்திரயான்', 'தென்', 'துருவம்', 'தரையிறங்கு', 'தயாராகி', 'வரு']]


In [57]:
stop_free_query_roots_joined = join_word_if_illai(stop_free_query_roots)
stop_free_sentences_roots_joined = join_word_if_illai(stop_free_sentences_roots)
print(stop_free_query_roots_joined)
print(stop_free_sentences_roots_joined)

[['சந்திரயான்', 'தென்', 'துருவம்', 'தரையிறங்கு']]
[['தென்', 'துருவம்', 'தரையிறங்கு', 'இந்தியா'], ['சந்திரயான்', 'நிலவு', 'தென்', 'துருவம்', 'ஆக்சிஜன்', 'கண்டுபிடிப்பு'], ['சந்திரயான்', 'திட்டம்', 'வெற்றி', 'நிலவு', 'தென்', 'துருவம்', 'கால்நடை', 'நாடு', 'இந்தியா'], ['வாரம்', 'சந்திரயான்', 'தென்', 'துருவம்', 'தரையிறங்கு', 'தயாராகி', 'வரு']]


In [58]:
query_proper_nouns = get_proper_nouns(stop_free_query_roots_joined)
sentences_proper_nouns = get_proper_nouns(stop_free_sentences_roots_joined)
print(query_proper_nouns)
print(sentences_proper_nouns)

[['சந்திரயான்']]
[[], ['சந்திரயான்', 'ஆக்சிஜன்'], ['சந்திரயான்'], ['சந்திரயான்', 'தயாராகி']]


In [59]:
query_proper_nouns_without_common_items = []
sentences_proper_nouns_without_common_items = []

for sentence in sentences_proper_nouns:
    sentences_proper_nouns_without_common_items.append(
        list(set(sentence) - set(query_proper_nouns[0]))
    )

for query in query_proper_nouns:
    qset = set(query)
    for sentence in sentences_proper_nouns:
        qset = qset - set(sentence)
    query_proper_nouns_without_common_items.append(list(qset))

print(sentences_proper_nouns_without_common_items)
print(query_proper_nouns_without_common_items)

[[], ['ஆக்சிஜன்'], [], ['தயாராகி']]
[[]]


In [60]:
query_proper_nouns_merged = [" ".join(query) for query in query_proper_nouns_without_common_items]
sentences_proper_nouns_merged =  [" ".join(sentence) for sentence in sentences_proper_nouns_without_common_items]

# query_vector = get_sentence_vectors(query_proper_nouns_merged)
# sentences_vector = get_sentence_vectors(sentences_proper_nouns_merged)


In [61]:
proper_noun_scores = []
for vector in sentences_proper_nouns_merged:
    score = fuzz.ratio(vector, query_proper_nouns_merged[0])
    proper_noun_scores.append(score * 0.01 or 0.5)

print(proper_noun_scores)

[1.0, 0.5, 1.0, 0.5]


In [62]:
query_without_proper_nouns = []
sentences_without_proper_nouns = []
for i in range(len(stop_free_sentences_roots_joined)):
    sentences_without_proper_nouns.append(list(set(stop_free_sentences_roots_joined[i]) - set(sentences_proper_nouns[i])))
query_without_proper_nouns.append(list(set(stop_free_query_roots_joined[0]) - set(query_proper_nouns[0])))

print(query_without_proper_nouns)
print(sentences_without_proper_nouns)

[['துருவம்', 'தென்', 'தரையிறங்கு']]
[['துருவம்', 'இந்தியா', 'தென்', 'தரையிறங்கு'], ['நிலவு', 'தென்', 'துருவம்', 'கண்டுபிடிப்பு'], ['தென்', 'வெற்றி', 'நாடு', 'நிலவு', 'துருவம்', 'இந்தியா', 'கால்நடை', 'திட்டம்'], ['தென்', 'தரையிறங்கு', 'வரு', 'துருவம்', 'வாரம்']]


In [63]:
sentence_opps = []
query_opps = []

for j, sentence in enumerate(sentences_without_proper_nouns, 0):
    opps = []
    for k, sword in enumerate(sentence, 0):
        for i, qword in enumerate(query_without_proper_nouns[0], 0):
            if(is_opposite(sword, qword)):
                print(sword, qword, "opposites")
                opps.append(sentences_without_proper_nouns[j][k])
                query_opps.append(query_without_proper_nouns[0][i])

    sentence_opps.append(opps)
    
query_opps = [list(set(query_opps))]

print(sentence_opps)
print(query_opps)


துருவம் துருவம்
துருவம் தென்
துருவம் தரையிறங்கு
இந்தியா துருவம்
இந்தியா தென்
இந்தியா தரையிறங்கு
தென் துருவம்
தென் தென்
தென் தரையிறங்கு
தரையிறங்கு துருவம்
தரையிறங்கு தென்
தரையிறங்கு தரையிறங்கு
நிலவு துருவம்
நிலவு தென்
நிலவு தரையிறங்கு
தென் துருவம்
தென் தென்
தென் தரையிறங்கு
துருவம் துருவம்
துருவம் தென்
துருவம் தரையிறங்கு
கண்டுபிடிப்பு துருவம்
கண்டுபிடிப்பு தென்
கண்டுபிடிப்பு தரையிறங்கு
தென் துருவம்
தென் தென்
தென் தரையிறங்கு
வெற்றி துருவம்
வெற்றி தென்
வெற்றி தரையிறங்கு
நாடு துருவம்
நாடு தென்
நாடு தரையிறங்கு
நிலவு துருவம்
நிலவு தென்
நிலவு தரையிறங்கு
துருவம் துருவம்
துருவம் தென்
துருவம் தரையிறங்கு
இந்தியா துருவம்
இந்தியா தென்
இந்தியா தரையிறங்கு
கால்நடை துருவம்
கால்நடை தென்
கால்நடை தரையிறங்கு
திட்டம் துருவம்
திட்டம் தென்
திட்டம் தரையிறங்கு
தென் துருவம்
தென் தென்
தென் தரையிறங்கு
தரையிறங்கு துருவம்
தரையிறங்கு தென்
தரையிறங்கு தரையிறங்கு
வரு துருவம்
வரு தென்
வரு தரையிறங்கு
துருவம் துருவம்
துருவம் தென்
துருவம் தரையிறங்கு
வாரம் துருவம்
வாரம் தென்
வாரம் தரையிறங்கு
[[], [], [], []]
[[]]


In [64]:
query_without_opposites = []
sentences_without_opposites = []
for i in range(len(sentences_without_proper_nouns)):
    sentences_without_opposites.append(list(set(sentences_without_proper_nouns[i]) - set(sentence_opps[i])))
query_without_opposites.append(list(set(query_without_proper_nouns[0]) - set(query_opps[0])))

print(query_without_opposites)
print(sentences_without_opposites)

[['துருவம்', 'தென்', 'தரையிறங்கு']]
[['துருவம்', 'தரையிறங்கு', 'தென்', 'இந்தியா'], ['நிலவு', 'தென்', 'துருவம்', 'கண்டுபிடிப்பு'], ['தென்', 'வெற்றி', 'நாடு', 'நிலவு', 'துருவம்', 'இந்தியா', 'கால்நடை', 'திட்டம்'], ['துருவம்', 'வாரம்', 'தென்', 'தரையிறங்கு', 'வரு']]


In [65]:
query_without_opposites_merged = [" ".join(query) for query in query_without_opposites]
sentences_without_opposites_merged =  [" ".join(sentence) for sentence in sentences_without_opposites]

print(query_without_opposites_merged)
print(sentences_without_opposites_merged)

query_without_opposites_vector = get_sentence_vectors(query_without_opposites_merged)
sentences_without_opposites_vector = get_sentence_vectors(sentences_without_opposites_merged)

without_opposites_scores = []
for vector in sentences_without_opposites_vector:
    print(score := cosine_similarity(vector, query_without_opposites_vector[0]))
    without_opposites_scores.append(score)

['துருவம் தென் தரையிறங்கு']
['துருவம் தரையிறங்கு தென் இந்தியா', 'நிலவு தென் துருவம் கண்டுபிடிப்பு', 'தென் வெற்றி நாடு நிலவு துருவம் இந்தியா கால்நடை திட்டம்', 'துருவம் வாரம் தென் தரையிறங்கு வரு']
0.9741255
0.84239554
0.7459006
0.76422


In [66]:
opposite_weights = []
influencer = 0.1
for opps_list in sentence_opps:
    if(len(opps_list) == 0):
        opposite_weights.append(1)
    else:
        opposite_weights.append(influencer * len(opps_list))

print(opposite_weights)

[1, 1, 1, 1]


In [67]:
print(proper_noun_scores)
print(without_opposites_scores)
print(opposite_weights)

[1.0, 0.5, 1.0, 0.5]
[0.9741255, 0.84239554, 0.7459006, 0.76422]
[1, 1, 1, 1]


In [68]:
authenticity_measure = np.mean([((x + y) / 2) * z for x, y, z in zip(proper_noun_scores, without_opposites_scores, opposite_weights)], axis = -1)
authenticity_measure

0.7908302024006844

In [75]:
import re

def keep_tamil_unicode(text):
    tamil_pattern = re.compile(r'[\u0B80-\u0BFF ]')
    tamil_text = ''.join(tamil_pattern.findall(text))
    return tamil_text

input_text = '''#CelebrityClicks | நடிகர் அஜித்குமாருக்கு DUCATI பைக்கை பிறந்தநாள் பரிசாக வழங்கிய ஷாலினி..!
                            #SunNews | #HBDAjithKumar | #AjithKumar | #ShaliniAjithkuma'''
tamil_only_text = keep_tamil_unicode(input_text)
print(tamil_only_text)

  நடிகர் அஜித்குமாருக்கு  பைக்கை பிறந்தநாள் பரிசாக வழங்கிய ஷாலினி                                  
